In [14]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from xbbg import blp
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

path = 'MBS_DATA'

def read_tba_prices(tba_symbol = "FNCL", coupon= 4):
    """
    Read TBA prices from the stored CSV file.
    
    :param coupon: Coupon rate to differentiate storage location
    :return: Pandas DataFrame with indexed dates and TBA prices
    """
    
    file_path = f"{path}/{tba_symbol}/{coupon:.1f}/prices.csv"
    if os.path.exists(file_path):
        return pd.read_csv(file_path, index_col=0, parse_dates=True)
    else:
        print(f"File {file_path} does not exist.")
        return None
    

def read_settle_dates(tba_symbol = 'FNCL'):
    """
    Read settle dates from the saved CSV file and return as a pandas Series.

    :return: Pandas Series with TBA names as index and settlement dates as values.
    """
    settle_dates_path = f"{path}/{tba_symbol}/settle_dates.csv"
    if os.path.exists(settle_dates_path):
        df = pd.read_csv(settle_dates_path, index_col=0, parse_dates=[1])
        return df.iloc[:, 0]  # Convert DataFrame column to Series
    else:
        raise FileNotFoundError(f"Settle dates file not found at {settle_dates_path}")
 

def read_mtg_rate():
    mtg_rates = pd.read_csv(f'{path}/mtg_rate.csv', index_col=0, parse_dates=True)
    return mtg_rates['US 30Y FRM'], mtg_rates['US 15Y FRM'] 


def read_repo():
    repo_rate = pd.read_csv(f'{path}/repo_ffund.csv', index_col=0, parse_dates=True).iloc[:,0]
    return repo_rate


def update_calendar():
    """
    Update the repo file with FEDL01 Index rates up to today using Bloomberg.
    Only fetches missing dates and does nothing if already up to date.
    
    Args:
        path (str): Directory path to the 'repo_ffund.csv' file.
    
    Returns:
        pd.Series: Updated repo rate series.
    """
    today = pd.Timestamp.today().normalize()
    
    # Load existing repo series
    repo_series = read_repo()
    last_date = repo_series.index[-1].normalize()
    
    if last_date >= today:
        print("Repo series is already up to date.")
        return None

    start_date = (last_date + timedelta(days=1)).strftime('%Y-%m-%d')
    end_date = today.strftime('%Y-%m-%d')
    
    # Fetch only missing data from Bloomberg
    new_data = blp.bdh(
        'FEDL01 Index',
        'PX_LAST',
        start_date,
        end_date,
        calendarCodeOverride='US'
    )

    if len(new_data) == 0 or new_data is None:
        print("No new data to add.")
        return repo_series

    # Append new data and save
    new_data = new_data.iloc[:,0]
    new_data.index = pd.to_datetime(new_data.index)
    updated_series = pd.concat([repo_series, new_data])
    updated_series.to_csv(f'{path}/repo_ffund.csv')

    print(f"Repo series updated with {len(new_data)} new records.")
    return None

################

# Global variable CALENDAR
#update_calendar()
CALENDAR = pd.Series(read_repo().index)


def next_business_day(date):
    """Return the next business day following the given date."""
    future_dates = CALENDAR[CALENDAR > date]
    if future_dates.empty:
        #raise ValueError("No future business day available.")
        return date + pd.offsets.BDay(1)
    return future_dates.iloc[0]


def prev_business_day(date):
    """Return the previous business day before the given date."""
    past_dates = CALENDAR[CALENDAR < date]
    if past_dates.empty:
        #raise ValueError("No previous business day available.")
        return date + pd.offsets.BDay(-1)
    return past_dates.iloc[-1]


def plus_bd(date, n):
    """Add or subtract n business days from date using CALENDAR."""
    if date > CALENDAR.values[-1] or date < CALENDAR.values[0]:
        return date + pd.offsets.BDay(n)
    
    if date not in CALENDAR.values:
        date = next_business_day(date)

    idx = CALENDAR[CALENDAR == date].index[0]
    new_idx = idx + n
    if new_idx < 0 or new_idx >= len(CALENDAR):
        #raise IndexError("Date out of bounds in calendar.")
        return date + pd.offsets.BDay(n)
    return CALENDAR.iloc[new_idx]


def accrued_interest(date: pd.Timestamp, coupon: float) -> float:
    """
    Calculate accrued interest for an MBS using Actual/Actual between payments.

    Args:
        date (pd.Timestamp): Settlement date.
        coupon (float): Annual coupon rate (e.g., 5.5 for 5.5%).

    Returns:
        float: Accrued interest per $100 face value.
    """
    coupon = coupon/100
    if date.day >= 25:
        prev_coupon_month = date.month
        prev_coupon_year = date.year
    else:
        prev_coupon_month = date.month - 1 or 12
        prev_coupon_year = date.year if date.month != 1 else date.year - 1

    tentative_prev_coupon = pd.Timestamp(prev_coupon_year, prev_coupon_month, 25)
    prev_coupon_date = (next_business_day(tentative_prev_coupon)
                        if tentative_prev_coupon not in CALENDAR.values
                        else tentative_prev_coupon)

    # Next payment is simply the 25th of the next month
    next_coupon_month = prev_coupon_date.month % 12 + 1
    next_coupon_year = prev_coupon_date.year + (prev_coupon_date.month // 12)
    tentative_next_coupon = pd.Timestamp(next_coupon_year, next_coupon_month, 25)
    next_coupon_date = (next_business_day(tentative_next_coupon)
                        if tentative_next_coupon not in CALENDAR.values
                        else tentative_next_coupon)

    # Actual/Actual between payments
    days_since_prev = (date - prev_coupon_date).days
    days_between_coupons = (next_coupon_date - prev_coupon_date).days

    accrued_interest = (coupon / 12) * (days_since_prev / days_between_coupons)

    return round(accrued_interest, 8)


def get_dates(t, T):
    """
    Generate an array of dates starting with t, ending with T, and all payment dates (25th of each month)
    in between, adjusted to next business day if needed.
    """
    dates = [t]
    current_date = t

    # First payment date: the 25th of t's month
    first_payment_date = pd.Timestamp(t.year, t.month, 25)
    if first_payment_date < t:
        # Move to the 25th of the next month
        year = t.year + (t.month // 12)
        month = (t.month % 12) + 1
        first_payment_date = pd.Timestamp(year, month, 25)

    # Adjust to next business day if needed
    if first_payment_date not in CALENDAR.values:
        first_payment_date = next_business_day(first_payment_date)

    # Add all payment dates until T
    while first_payment_date < T:
        dates.append(first_payment_date)
        year = first_payment_date.year + (first_payment_date.month // 12)
        month = (first_payment_date.month % 12) + 1
        next_payment_date = pd.Timestamp(year, month, 25)
        if next_payment_date not in CALENDAR.values:
            next_payment_date = next_business_day(next_payment_date)
        first_payment_date = next_payment_date

    dates.append(T)

    return np.array(dates, dtype='datetime64[D]')



In [ ]:
tba_symbol = 'FNCL'
ALL_CPNS = np.arange(2.5, 7.5, 0.5)
prices = read_tba_prices(tba_symbol=tba_symbol, coupon=4)
current = pd.DataFrame(index = prices.index, columns = ALL_CPNS, dtype=float)
current_returns = pd.DataFrame(index = prices.index, columns = ALL_CPNS, dtype=float)


for coupon in ALL_CPNS:
    settle_dates = read_settle_dates(tba_symbol)
    settle_dates.index = settle_dates.apply(lambda x: f"{tba_symbol} {coupon} {x.strftime('%m/%y')} Mtge")

    prices = read_tba_prices(tba_symbol=tba_symbol, coupon=coupon)
    returns = read_tba_prices(tba_symbol=tba_symbol, coupon=coupon).diff().shift(-1)
    d0 = prices.index[0]

    for tba in prices.columns :
        set_date = settle_dates.loc[tba]
        prev_set_date = d0 if tba==settle_dates.index[0] else  settle_dates.loc[:tba].iloc[-2]
        start = max(plus_bd(prev_set_date, -1), d0)
        end = plus_bd(set_date, -1)
        current.loc[start:end, coupon] = prices.loc[start:end, tba].values
        current_returns.loc[start:end, coupon] = returns.loc[start:end, tba].values

In [18]:
prices.columns[0]

'FNCL 2 04/20 Mtge'

In [12]:
current.to_csv('../tba_rv/Data/rolled_tba.csv')

In [13]:
current_returns.to_csv('../tba_rv/Data/rolled_tba_returns.csv')

In [16]:
prices

,FNCL 2 04/20 Mtge,FNCL 2 05/20 Mtge,FNCL 2 06/20 Mtge,FNCL 2 08/20 Mtge,FNCL 2 09/20 Mtge,FNCL 2 10/20 Mtge,FNCL 2 02/21 Mtge,FNCL 2 03/21 Mtge,FNCL 2 04/21 Mtge,FNCL 2 05/21 Mtge,...,FNCL 2 09/24 Mtge,FNCL 2 10/24 Mtge,FNCL 2 11/24 Mtge,FNCL 2 12/24 Mtge,FNCL 2 01/25 Mtge,FNCL 2 02/25 Mtge,FNCL 2 03/25 Mtge,FNCL 2 04/25 Mtge,FNCL 2 05/25 Mtge,FNCL 2 06/25 Mtge
2019-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,79.531250,79.562500,79.531250,NaN
2025-03-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,79.390625,79.359375,79.375000,NaN
2025-03-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,79.929688,79.867188,79.890625,NaN
2025-03-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,79.421875,79.367188,79.382812,79.375000
